In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns

from agents import RassalAjan, UCBAjan, Ortam, ThompsonAjan, SentetikOrtam

from ipywidgets import interact

# Upper Confidence Bound Agent

In [2]:
@interact(tur_sayisi=(1,750,1))
def ucb(tur_sayisi=95):
    ortam = Ortam('Ads_CTR_Optimisation.csv')
#     ortam = SentetikOrtam([0.4, 0.2, 0.3, 0.05])
    ucb = UCBAjan(ortam.n_ads)
    
    for _ in range(tur_sayisi):
        ucb.ogren(ortam)
#     print("Toplam odul:", ucb.toplam_odul())
    df = pd.DataFrame(ucb.tablo).T.assign(u=lambda df: df['UCB'] - df['ri'])
    df.plot(kind='bar', y=['ri', 'u'], stacked=True, figsize=(12,8))
    plt.axhline((df['ri'] + df['u']).max(), label='max ucb', color='red')
    plt.legend()
    plt.ylim(0, 1)
#     plt.xlim(-1, 11)
    

interactive(children=(IntSlider(value=95, description='tur_sayisi', max=750, min=1), Output()), _dom_classes=(…

# Thompson Sampler Agent

In [3]:
# ortam = Ortam('Ads_CTR_Optimisation.csv')
ortam = SentetikOrtam([0.1, 0.3, 0.4])
# ortam.data = ortam.data.iloc[:, :4]
# ortam.n_ads = 4
N = 0
thomp = ThompsonAjan(ortam.n_ads)
@interact(tur_sayisi=(1,1000,5))
def thompson(tur_sayisi=10):
    global N, thomp
#     ortam.yenile()
    
    for _ in range(tur_sayisi):
        if _ < N:
            continue
        N += 1
        thomp.ogren(ortam)
    dists = thomp.tablo_goruntule().T.apply(lambda x: np.random.beta(x['Ni1'] + 1, x['Ni0'] + 1, size=50000), axis=1)
#     print(_)
#     print(N)
    plt.figure(figsize=(16, 4))
    for d in dists.index:
        sns.kdeplot(dists[d], label=f'Ad {d}')
    plt.legend()
    plt.ylim(0, 18)
    plt.xlim(-0.2, 0.9)

interactive(children=(IntSlider(value=10, description='tur_sayisi', max=1000, min=1, step=5), Output()), _dom_…